<a href="https://colab.research.google.com/github/hmin27/2023_DL_Clip/blob/main/fewshot_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil
import random

def create_shot_directories(shots, random_seed=42):

    random.seed(random_seed)

    for shot in shots:
        shot_dir = os.path.join(few_shot_dataset_path, f"{shot}_shot")
        os.makedirs(shot_dir)

        for split in ['train', 'validation', 'test']:
            split_dir = os.path.join(shot_dir, split)
            os.makedirs(split_dir)

            for class_name in os.listdir(full_dataset_path):
                class_dir = os.path.join(full_dataset_path, class_name)
                class_samples = os.listdir(class_dir)

                random.shuffle(class_samples)

                split_size = shot

                if split == 'train':
                    selected_samples = class_samples[:split_size]
                elif split == 'validation':
                    selected_samples = class_samples[split_size:split_size * 2]
                else:
                    selected_samples = class_samples[split_size * 2:split_size * 3]

                class_split_dir = os.path.join(split_dir, class_name)
                os.makedirs(class_split_dir)

                for sample in selected_samples:
                    src_path = os.path.join(class_dir, sample)
                    dest_path = os.path.join(class_split_dir, sample)
                    shutil.copy(src_path, dest_path)

full_dataset_path = '/content/drive/MyDrive/Study/DL_CLIP/Food'
few_shot_dataset_path = '/content/drive/MyDrive/Study/DL_CLIP/Food_fewshot'

shots = [1, 2, 4, 8, 16]
create_shot_directories(shots)


In [ ]:
import os
import shutil
import random

def create_shot_directories(shots, full_dataset_path, few_shot_dataset_path, random_seed=42):
    random.seed(random_seed)

    for shot in shots:
        shot_dir = os.path.join(few_shot_dataset_path, f"{shot}_shot")
        os.makedirs(shot_dir)

        # 각 split에서 이미지 겹치지 않도록 클래스 별로 이미지 선정
        for split in ['train', 'validation', 'test']:
            split_dir = os.path.join(shot_dir, split)
            os.makedirs(split_dir)

            # 클래스 목록 생성
            class_list = os.listdir(full_dataset_path)

            for class_name in class_list:
                class_dir = os.path.join(full_dataset_path, class_name)
                class_samples = os.listdir(class_dir)

                random.shuffle(class_samples)

                split_size = shot

                # 클래스 별로 이미지 선정
                selected_samples = class_samples[:split_size]

                class_split_dir = os.path.join(split_dir, class_name)
                os.makedirs(class_split_dir)

                for sample in selected_samples:
                    src_path = os.path.join(class_dir, sample)
                    dest_path = os.path.join(class_split_dir, sample)
                    shutil.copy(src_path, dest_path)

        # test에서 샷 수만큼의 이미지만 들어가게 만들기
        test_dir = os.path.join(shot_dir, 'test')
        for class_name in class_list:
            class_split_dir = os.path.join(test_dir, class_name)

            # 샷 수만큼 이미지만 복사
            selected_samples = os.listdir(class_split_dir)[:shot]

            for sample in selected_samples:
                src_path = os.path.join(class_split_dir, sample)
                dest_path = os.path.join(test_dir, class_name, sample)
                shutil.copy(src_path, dest_path)

        # 어떤 샷에서는 전체 클래스가 아니라 클래스 중 일부만 들어가 있을 때 수정
        for split in ['train', 'validation', 'test']:
            split_dir = os.path.join(shot_dir, split)
            for class_name in class_list:
                class_split_dir = os.path.join(split_dir, class_name)

                # 존재하지 않는 클래스 디렉토리는 건너뜀
                if not os.path.exists(class_split_dir):
                    continue

                # 클래스 중 일부만 들어가 있을 경우, 클래스 전체로 복사
                if len(os.listdir(class_split_dir)) < shot:
                    for sample in os.listdir(class_split_dir):
                        src_path = os.path.join(class_split_dir, sample)
                        dest_path = os.path.join(split_dir, class_name, sample)
                        shutil.copy(src_path, dest_path)

full_dataset_path = '/content/drive/MyDrive/Study/DL_CLIP/Food'
few_shot_dataset_path = '/content/drive/MyDrive/Study/DL_CLIP/Food_fewshot'

shots = [1, 2, 4, 8, 16]
create_shot_directories(shots, full_dataset_path, few_shot_dataset_path)
